In [3]:
"""Evaluate a models testing keyrank at N traces and how many traces required to achieve 99% accuracy"""

import data
import sys
import numpy as np
import torch
import json

import training

import keyrank_rs


def get_best_epoch(model_name) -> int:
    with open(f"models/eval/{model_name}.txt", 'r') as f:
        lines = f.readlines()[1:] # skip "N traces" line
        lines = [float(line.strip()) for line in lines]
        best_epoch = np.array(lines).argmin()
    return best_epoch.item()


device = torch.device("cuda")

testing_data = {}

IMPL = "fixslice"
ARCH = "zhang"
PREDICTION_TARGET = "2sbox"
TARGET_BYTE_IDX = 1
TRACE_START = 0
TRACE_END = 1000
SEED = 777

LOSS_WEIGHT = 0.5

N_TRACES = 10

testing_data["n_traces"] = N_TRACES

_,_,test_loader = data.get_dataloaders(
    200,
    PREDICTION_TARGET,
    TARGET_BYTE_IDX,
    TRACE_START,
    TRACE_END,
)

In [6]:

model_name = f"{IMPL}-{PREDICTION_TARGET}-byte{TARGET_BYTE_IDX}-{ARCH}-{TRACE_START}_{TRACE_END}"
epoch = 44 #get_best_epoch(model_name)

#testing_data["best_epoch"] = epoch
model_path = f"models/{model_name}/epoch{epoch}.pt"

print(model_path)

model = torch.load(model_path)

model.eval()


with torch.no_grad():
    if PREDICTION_TARGET == "key":
        testing_keyrank = training.mean_keyrank(model, test_loader, N_TRACES)
    elif PREDICTION_TARGET == "sbox":
        testing_keyrank = training.mean_sbox_rank(model, test_loader, N_TRACES)
    elif PREDICTION_TARGET == "sbox2":
        testing_keyrank = training.mean_sbox_rank(model, test_loader, N_TRACES, plaintext=1)
    elif PREDICTION_TARGET in ["2sbox", "2sbox*", "2sbox..."]:
        testing_keyrank = training.mean_2sbox_rank(model, test_loader, N_TRACES)

testing_data["testing_keyrank"] = testing_keyrank
testing_keyrank

C:\Users\Ulrik\AppData\Local\Temp\ipykernel_13712\1842216167.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)
evaluating:   0%|          | 

models/fixslice-2sbox-byte1-zhang-0_1000/epoch44.pt


evaluating: 100%|██████████| 500/500 [00:01<00:00, 331.42it/s]


tensor(11.9730, device='cuda:0', dtype=torch.float64)

In [10]:
from tqdm import tqdm

def traces_for_99acc(model, test_loader):
    # maps (N traces, key) to success/failure
    success_matrix = np.zeros((500,500))

    for key_idx, (traces, plaintexts, true_key) in tqdm(enumerate(test_loader), unit='key'):

        # All 500 traces
        traces : torch.Tensor = traces.to(device).squeeze()
        plaintexts : torch.Tensor = plaintexts.to(device)

        # All 500 plaintexts
        plaintexts = plaintexts.long().detach().cpu().numpy().squeeze()
        plaintexts : np.ndarray  = plaintexts.transpose((1, 0))


        # 2PT model outputs list of 2 tensors
        sbox_scores = model(traces)
        if type(sbox_scores) is list:
            sbox_scores = torch.stack(sbox_scores).detach().cpu().numpy()
        else:
            sbox_scores = sbox_scores.detach().cpu().numpy()


        keyscores_both = []
        for scores, pt in zip(sbox_scores, plaintexts):

            numpy_keyscores = keyrank_rs.sbox_scores_to_keyscores_parallel(pt, scores)
            keyscores_both.append(torch.Tensor(numpy_keyscores).to(device))

        keyscores_both = [torch.Tensor(np_ks) for np_ks in keyscores_both]


        keyscore_acc = torch.zeros(256)

        for keyscores in keyscores_both:
            # logsum scores before calculating rank
            keyscores = keyscores.softmax(dim=1).log()

            for idx,keyscore in enumerate(keyscores):
                keyscore_acc += keyscore.cpu()
                rank1_value = keyscore_acc.argmax()
                success_matrix[idx, key_idx] = 1. if rank1_value == true_key else 0.

    N_traces_successes = success_matrix.sum(axis=0)

    for idx, n_hits in enumerate(N_traces_successes):
        if n_hits >= 495.0: # 99%
            return idx+1

    return -1


traces_needed = traces_for_99acc(model, test_loader)

500key [00:29, 16.71key/s]


In [11]:
traces_needed

1